In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import reshape
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dense, Dropout
from keras.utils import to_categorical

2024-11-03 16:54:21.111056: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Import Data 


In [2]:
path = R'/Users/shaquillelee/Documents/Specialization /Datasets'

In [3]:
weather_df = pd.read_csv(os.path.join(path, 'Prepared Data', 'weather_cleaned.csv'), index_col = 0)
pleasant_df = pd.read_csv(os.path.join(path, 'Prepared Data', 'pleasant_cleaned.csv'), index_col = 0)

In [4]:
weather_df.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [5]:
weather_df.shape

(22950, 135)

In [6]:
pleasant_df.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
pleasant_df.shape

(22950, 15)

# Split Data 

In [8]:
# Create 'X' Matrix 
X = weather_df
y = pleasant_df

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [11]:
X = X.reshape(-1,15,9)

In [12]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [13]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360, 15)
(4590, 15, 9) (4590, 15)


# Creating RNN

In [14]:
stations = {
    0: 'BASEL',
    1: 'BELGRADE',
    2: 'BUDAPEST',
    3: 'DEBILT',
    4: 'DUSSELDORF',
    5: 'HEATHROW',
    6: 'KASSEL',
    7: 'LJUBLJANA',
    8: 'MAASTRICHT',
    9: 'MADRID',
   10: 'MUNCHENB',
   11: 'OSLO',
   12: 'SONNBLICK',
   13: 'STOCKHOLM',
   14: 'VALENTIA',
}

In [15]:
# Create a Keras leyered model. Use initial hyperparameters: 4, 16, 18, softmax
epochs = 4
batch_size = 16
n_hidden = 18

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [17]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/4
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.0987 - loss: 9.7779 - val_accuracy: 0.0349 - val_loss: 8.6965
Epoch 2/4
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0798 - loss: 9.7906 - val_accuracy: 0.0296 - val_loss: 8.9405
Epoch 3/4
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0844 - loss: 9.8592 - val_accuracy: 0.0373 - val_loss: 9.2180
Epoch 4/4
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0840 - loss: 10.0351 - val_accuracy: 0.0375 - val_loss: 9.5386


In [18]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [19]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pred        BELGRADE  KASSEL  MADRID  STOCKHOLM  VALENTIA
True                                                     
BASEL              1       3    1246          0      1712
BELGRADE           2       0     831          1        48
BUDAPEST           1       0     155          0         4
DEBILT             0       0      74          0         1
DUSSELDORF         2       0      14          0         1
HEATHROW           1       0      67          0         7
KASSEL             0       0      16          0         0
LJUBLJANA          0       0      38          0         3
MAASTRICHT         0       0       8          0         1
MADRID             3       0     170          0       167
MUNCHENB           0       0       3          0         2
OSLO               0       0       5          0         0
STOCKHOLM          0       0       1          0         0
VALENTIA           0       0       2          0         0


In [20]:
# Create a Keras leyered model. Use initial hyperparameters: 30, 32, 64, softmax
epochs = 30
batch_size = 32
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax')) 

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [22]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0903 - loss: 11.4562 - val_accuracy: 0.0562 - val_loss: 9.3883
Epoch 2/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0801 - loss: 11.5894 - val_accuracy: 0.0386 - val_loss: 9.7370
Epoch 3/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0895 - loss: 11.6941 - val_accuracy: 0.0505 - val_loss: 9.6085
Epoch 4/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0823 - loss: 11.5776 - val_accuracy: 0.0405 - val_loss: 10.1398
Epoch 5/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0894 - loss: 12.1824 - val_accuracy: 0.0447 - val_loss: 10.4209
Epoch 6/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0846 - loss: 12.3048 - val_accuracy: 0.0451 - val_loss: 10.8237
Epoch 7/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0815 - loss: 12.9233 - val_accuracy: 0.0420 - val_loss: 11.0942
Epoch 8/30
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0810 - loss: 12.9845 - val_

In [23]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [24]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Pred        KASSEL  LJUBLJANA  MADRID  OSLO  SONNBLICK  VALENTIA
True                                                            
BASEL            1          0    1196    21          2      1742
BELGRADE         1          1     814    12          0        54
BUDAPEST         0          0     152     3          0         5
DEBILT           0          0      74     1          0         0
DUSSELDORF       0          0      17     0          0         0
HEATHROW         0          0      70     0          0         5
KASSEL           0          0      16     0          0         0
LJUBLJANA        0          0      38     2          0         1
MAASTRICHT       0          0       8     0          0         1
MADRID           1          0     212     6          0       121
MUNCHENB         0          0       2     0          0         3
OSLO             0          0       5     0          0         0
STOCKHOLM        0          0       1     0      

In [36]:
# Create a Keras leyered model. Change activation type: 30, 32, 64, sigmoid
epochs = 20
batch_size = 32
n_hidden = 84

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [38]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1087 - loss: 10.8501 - val_accuracy: 0.1887 - val_loss: 9.2777
Epoch 2/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0982 - loss: 11.6296 - val_accuracy: 0.0519 - val_loss: 9.5595
Epoch 3/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0911 - loss: 11.5146 - val_accuracy: 0.0420 - val_loss: 10.0103
Epoch 4/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0994 - loss: 12.0766 - val_accuracy: 0.0298 - val_loss: 10.5125
Epoch 5/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0887 - loss: 12.3604 - val_accuracy: 0.0479 - val_loss: 10.8587
Epoch 6/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0863 - loss: 12.5137 - val_accuracy: 0.0538 - val_loss: 11.4028
Epoch 7/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0909 - loss: 13.0606 - val_accuracy: 0.0453 - val_loss: 11.7446
Epoch 8/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0888 - loss: 13.1860 - val

In [39]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [40]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  MADRID
True                     
BASEL        2962       0
BELGRADE      882       0
BUDAPEST      160       0
DEBILT         75       0
DUSSELDORF     17       0
HEATHROW       75       0
KASSEL         16       0
LJUBLJANA      41       0
MAASTRICHT      9       0
MADRID        338       2
MUNCHENB        5       0
OSLO            5       0
STOCKHOLM       1       0
VALENTIA        2       0


In [41]:
# Create a Keras leyered model. Change activation type: 30, 32, 64, tanh
epochs = 20
batch_size = 32
n_hidden = 84

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh'))

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [42]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [43]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0814 - loss: 24.8048 - val_accuracy: 0.1237 - val_loss: 21.4896
Epoch 2/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0891 - loss: 25.1021 - val_accuracy: 0.0111 - val_loss: 24.0170
Epoch 3/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0323 - loss: 25.1811 - val_accuracy: 2.1786e-04 - val_loss: 20.5554
Epoch 4/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0206 - loss: 24.9426 - val_accuracy: 8.7146e-04 - val_loss: 29.7011
Epoch 5/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0092 - loss: 24.4116 - val_accuracy: 4.3573e-04 - val_loss: 25.5055
Epoch 6/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0128 - loss: 24.6898 - val_accuracy: 6.5359e-04 - val_loss: 26.0767
Epoch 7/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0336 - loss: 24.7053 - val_accuracy: 2.1786e-04 - val_loss: 22.7283
Epoch 8/20
574/574 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0253

In [44]:
# Create confusion matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [45]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Pred        BELGRADE  DUSSELDORF  MAASTRICHT  STOCKHOLM
True                                                   
BASEL           1321          50          11       1580
BELGRADE          45           0           0        837
BUDAPEST           5           0           0        155
DEBILT             0           0           0         75
DUSSELDORF         0           0           0         17
HEATHROW           3           0           0         72
KASSEL             0           0           0         16
LJUBLJANA          3           1           0         37
MAASTRICHT         1           0           0          8
MADRID            81           6           0        253
MUNCHENB           0           0           0          5
OSLO               0           0           0          5
STOCKHOLM          0           0           0          1
VALENTIA           0           0           0          2
